**--- Part One ---**

In [1]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
case = "jfm"
case = "gatton"
# 
# verbose output
verbose = False

import numpy as np
from itertools import cycle

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

import numpy as np
import multiprocessing as mp
from tqdm.notebook import tqdm

class Sensor():
    def __init__(self, string):
        string = string.strip()
        s,b = string.split(':')
        
        s = ''.join(s.split(' ')[-2:])
        sj, si = s.split(',')
        si = int(si.split('=')[-1])
        sj = int(sj.split('=')[-1])

        b = ''.join(b.split(' ')[-2:])
        bj, bi = b.split(',')
        bi = int(bi.split('=')[-1])
        bj = int(bj.split('=')[-1])
        
        self.pos = np.array([si,sj])
        self.beacon = np.array([bi,bj])
        self.dist = abs(si-bi) + abs(sj-bj)
        self.rangei = [self.pos[0]-self.dist, self.pos[0]+self.dist]
        self.rangej = [self.pos[1]-self.dist, self.pos[1]+self.dist]

    def update_pos(self,delta):
        self.pos -= delta
        self.beacon -= delta
        self.rangei = [self.pos[0]-self.dist, self.pos[0]+self.dist]
        self.rangej = [self.pos[1]-self.dist, self.pos[1]+self.dist]
    
    def update_outside_range(self):
        i,j = self.pos
        dist = self.dist
        points = [[i+dist+1, j],[i-dist-1, j]]
        print(f'updating sensor: {self.pos}')
        for i in tqdm(range(self.rangei[0], self.rangei[1]+1)):
            delta = dist - abs(self.pos[0]-i)+1
            points.append([i,j-delta])
            points.append([i,j+delta])
        self.outside_range = points

conv = {0:'.', 1:'#', 2:'S', 3:'B'}

class Grid():
    def __init__(self, start=None):
        self.finished = False
        self.sensors = []
        for line in input_lines:
            self.sensors.append(Sensor(line.strip()))
        self.imin = min([s.rangei[0] for s in self.sensors])
        self.jmin = min([s.rangej[0] for s in self.sensors])
        self.imax = max([s.rangei[1] for s in self.sensors])
        self.jmax = max([s.rangej[1] for s in self.sensors])

        # # print(f'({imin},{jmin}) to ({imax},{jmax})')
        # deli = self.imax-self.imin
        # delj = self.jmax-self.jmin

        #self.grid = np.zeros([deli+1,delj+1])

        for s in self.sensors:
        #     # print(f'{s.pos} -> {s.pos+(imin,jmin)}')
            s.update_pos((self.imin,self.jmin))
            s.update_outside_range()
            # s.pos -= (self.imin,self.jmin)
            # s.beacon -= (self.imin,self.jmin)
        #     self.grid[tuple(s.pos)] = 2
        #     self.grid[tuple(s.beacon)] = 3

    def covered_by(self, pos, sensor):
        dist = abs(sensor.pos[0]-pos[0]) + abs(sensor.pos[1]-pos[1])
        # print(f'{pos} <-> {sensor.pos}: {dist} <= {sensor.dist}')
        return dist <= sensor.dist

    def check_row(self, row_idx):
        #row_idx -= self.imin
        # print(f'{row_idx=}')
        row = np.zeros([self.jmax-self.jmin+1])
        # print(len(self.sensors))
        sensors = [s for s in self.sensors if s.rangei[0]<=row_idx<=s.rangei[1]]
        # print(len(sensors))
        for s in sensors:
            vert_dist = abs(s.pos[0]-row_idx)
            row_dist = abs(s.dist-vert_dist)
            start = s.pos[1]-row_dist
            end = s.pos[1]+row_dist+1
            row[start:end] = 1
            if s.pos[0] == row_idx: row[s.pos[1]] = 2
            if s.beacon[0] == row_idx: row[s.beacon[1]] = 3
            # print(f'{vert_dist=}, {row_dist=}, {start=}, {end=}')
        for s in sensors:
            # print(f'sensor: {s.pos}; beacon: {s.beacon}')
            if s.pos[0] == row_idx: row[s.pos[1]] = 2
            if s.beacon[0] == row_idx:
                # print(f'added beacon: {s.beacon[1]}') 
                row[s.beacon[1]] = 3
        return row
    
    def check_row_for_open(self, row_idx):
        print(f'checking row: {row_idx}')
        if self.finished: 
            return 0
        row = self.check_row(row_idx)[self.start[1]:self.end[1]+1]
        search = np.where(row==0)
        if len(search[0]):
            y = row_idx + self.imin
            x = row_idx + self.jmin +1
            freq = 4000000*x + y
            print(f'{x=}, {y=}: {freq}')
            self.finished = True
            return freq
        return 0

    def count_blocked(self, row_idx):
        row = self.check_row(row_idx)
        search = np.where(row == 1)
        return len(search[0])
    
    def find_option_parallel(self, max_idx=20):
        pool = mp.Pool(mp.cpu_count()-1)
        self.start = np.array((0,0)) - (self.imin,self.jmin)
        self.end = np.array((max_idx,max_idx)) - (self.imin,self.jmin)
        results = [pool.apply_async(self.check_row_for_open, args=(row_idx)) for row_idx in range(self.start[0],self.end[0]+1)]
        results = [r.get()[1] for r in results]
        pool.close()
        # results = [pool.apply(self.check_row_for_open, args=(row_idx, start, end)) for row_idx in range(start[0],end[0]+1)]
        return results

    def find_option(self, max_idx=20):
        start = np.array((0,0)) - (self.imin,self.jmin)
        self.start = start
        end = np.array((max_idx,max_idx)) - (self.imin,self.jmin)
        self.end = end
        n_show = 100
        n_rows = end[0]-start[0]
        print(f'searching rows: {start[0]} through {end[0]}')
        for row_idx in range(start[0],end[0]+1):
            if not row_idx%n_show:
                print(f'checking row: {row_idx} ({(row_idx-start[0])/(end[0]-start[0])*100}%)')
            # self.check_row_for_open(row_idx)
            row = self.check_row(row_idx)[start[1]:end[1]+1]
            # print(f'row: {row_idx}\n{row}')
            search = np.where(row==0)
            if len(search[0]):
                y = row_idx + self.imin
                x = row_idx + self.jmin +1
                freq = 4000000*x + y
                print(f'{x=}, {y=}: {freq}')
                return freq


    # def check_row(self, row_idx):
    #     row_idx -= self.imin
    #     for j,val in enumerate(self.grid[row_idx,:]):
    #         if val > 0:
    #             continue
    #         pos = np.array((row_idx, j))
    #         # print(f'checking: {pos}')
    #         for s in self.sensors:
    #             # print(f'    against: {s.pos}')
    #             if self.covered_by(pos, s):
    #                 self.grid[tuple(pos)] = 1
    #                 break

    # def count_row(self, row_idx):
    #     row_idx -= self.imin
    #     search = np.where(self.grid[row_idx,:]==1)
    #     return len(search[0])

    def show(self):
        for row in self.grid:
            print(f"{''.join([conv[val] for val in row])}")
        # self.grid = np.array([[abc[val] for val in row.strip()] for row in input_lines])
        # if start is not None:
        #     self.start = np.array(start)

In [2]:
grid = Grid()
row = 2000000 - grid.imin
# row = 10 - grid.imin
grid.count_blocked(row)

updating sensor: [3854619 5612147]


  0%|          | 0/776261 [00:00<?, ?it/s]

updating sensor: [4308102 2364317]


  0%|          | 0/2253067 [00:00<?, ?it/s]

updating sensor: [3736672 1818087]


...

updating sensor: [4477128 3601330]


  0%|          | 0/2573669 [00:00<?, ?it/s]

updating sensor: [4184444 5286254]


  0%|          | 0/1281513 [00:00<?, ?it/s]

4665948

**--- Part Two ---**

In [ ]:
#grid = Grid()
print('grid built')
possible_locs = set()
maxidx = 4000000 # Actual
mini,minj = np.array((0,0)) - (grid.imin, grid.jmin)
maxi,maxj = np.array((maxidx, maxidx)) - (grid.imin, grid.jmin) 
for s in tqdm(grid.sensors):
    # print(f'building points for sensor {s.pos}')
    for point in s.outside_range:
        possible_locs.add(tuple(point))
print('checking points')
for point in tqdm(possible_locs):
    if (point[0]<mini or point[1]<minj or
        point[0]>maxi or point[1]>maxj):
        # print('out of range')
        continue
    covered = False
    for s in tqdm(grid.sensors):
        if grid.covered_by(point, s):
            # print(f'{point} covered')
            covered = True
            break
    if covered == False:
        y = point[0] + grid.imin
        x = point[1] + grid.jmin 
        freq = 4000000*x + y
        print(f'{x=}, {y=}: {freq}')
        print(point)
        break

In [5]:
print('checking points')
for point in tqdm(possible_locs):
    if (point[0]<mini or point[1]<minj or
        point[0]>maxi or point[1]>maxj):
        # print('out of range')
        continue
    covered = False
    for s in grid.sensors:
        if grid.covered_by(point, s):
            # print(f'{point} covered')
            covered = True
            break
    if covered == False:
        y = point[0] + grid.imin
        x = point[1] + grid.jmin 
        freq = 4000000*x + y
        print(f'{x=}, {y=}: {freq}')
        print(point)
        break

checking points


  0%|          | 0/82798166 [00:00<?, ?it/s]

x=3385922, y=2671045: 1658786757
(3755385, 5074556)


C:\Users\jackm\AppData\Local\Temp/ipykernel_51828/1522754895.py:16: RuntimeWarning: overflow encountered in long_scalars
  freq = 4000000*x + y


In [ ]:
# JFM
x=2889605
y=3398893
sx = str(x*4)
sy = str(y)
print(len(sy))
print(sx[-1])
if len(sy)>6 and sx[-1]=='0':
    s = sx[:-1]+sy
s

In [12]:
# Gatton
x=3385922
y=2671045
sx = str(x*4)
sy = str(y)
print(len(sy))
print(sx[-1])
print(sx)
print(sy)
if len(sy)>6 and sx[-1]=='0':
    s = sx[:-1]+sy
else:
    sx = str(int(sx)+int(sy[0]))
    s = sx+sy[1:]
s

7
8
13543688
2671045


'13543690671045'

In [15]:
x=14
y=11
sy = str(y)
s = str(x*4)+'0'*(6-len(sy))+str(y)
s

'56000011'

In [11]:
grid = Grid()
max_idx = 20 # Example
max_idx = 4000000 # Actual
grid.find_option(max_idx)


searching rows: 1877632 through 5877632
checking row: 1877700 (0.0017%)
checking row: 1877800 (0.0042%)
checking row: 1877900 (0.0067%)
checking row: 1878000 (0.0092%)
checking row: 1878100 (0.0117%)
checking row: 1878200 (0.0142%)
checking row: 1878300 (0.0167%)
checking row: 1878400 (0.019200000000000002%)
checking row: 1878500 (0.0217%)
checking row: 1878600 (0.0242%)
checking row: 1878700 (0.026699999999999998%)
checking row: 1878800 (0.0292%)
checking row: 1878900 (0.0317%)
checking row: 1879000 (0.0342%)
checking row: 1879100 (0.036699999999999997%)
checking row: 1879200 (0.0392%)
checking row: 1879300 (0.0417%)
checking row: 1879400 (0.0442%)
checking row: 1879500 (0.046700000000000005%)
checking row: 1879600 (0.0492%)
checking row: 1879700 (0.051699999999999996%)
checking row: 1879800 (0.0542%)
checking row: 1879900 (0.0567%)
checking row: 1880000 (0.059199999999999996%)
checking row: 1880100 (0.061700000000000005%)
checking row: 1880200 (0.0642%)
checking row: 1880300 (0.0667%

KeyboardInterrupt: 

In [3]:
grid = Grid()
max_idx = 20 # Example
# max_idx = 4000000 # Actual
results = grid.find_option_parallel(max_idx)

